In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\wine_quality_predication_project\\research'

In [4]:
os.chdir("../")

In [22]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\wine_quality_predication_project'

In [15]:
from dataclasses import dataclass
from pathlib import Path

In [16]:
@dataclass(frozen=True)
class ModelEvaluationConfig: 
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [23]:
from ml_project.constants import *
from ml_project.utils.common import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 parmas_filepath= PARAMS_FILE_PATH,
                 schema_filepath =SCHEMA_FILE_PATH):
        self.config =read_yaml(config_filepath)
        self.parmas=read_yaml(parmas_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)->ModelEvaluationConfig:

        config=self.config.model_evaluation
        params=self.parmas.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name)
        

        return model_evaluation_config
        
        

In [30]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib
from ml_project.utils.common import save_json

In [40]:
class ModelEvaluation: 
    def __init__(self, config=ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rms=np.sqrt(mean_squared_error(actual, pred))
        mse=mean_absolute_error(actual, pred)
        r2=r2_score(actual, pred)

        return rms, mse, r2
    
    def save_results(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column], axis=1)
        test_y=test_data[[self.config.target_column]]

        predict_data=model.predict(test_x)

        (rms, mse, r2)=self.eval_metrics(actual=test_y, pred=predict_data)

        # Saving metrics path as local.

        score={"rms":rms, "mse": mse, "r2":r2}

        save_json(path=Path(self.config.metric_file_name),  data= score)



In [41]:
## Creating pipeline
try: 
    config=ConfigurationManager()
    model_evaluation_config=config.get_model_evaluation_config()
    model_evalutaion=ModelEvaluation(model_evaluation_config)
    model_evalutaion.save_results()

except Exception as e: 
    raise e

[2023-09-24 09:25:40,394 : INFO: common : Yaml file: config\config.yaml loaded sucessfully]
[2023-09-24 09:25:40,401 : INFO: common : Yaml file: params.yaml loaded sucessfully]
[2023-09-24 09:25:40,410 : INFO: common : Yaml file: schema.yaml loaded sucessfully]
[2023-09-24 09:25:40,413 : INFO: common : Created directory as : artifacts]
[2023-09-24 09:25:40,417 : INFO: common : Created directory as : artifacts/model_evaluation]
[2023-09-24 09:25:40,446 : INFO: common : Json file saved at: artifacts\model_evaluation\metrics.json]
